In [1]:
import pandas as pd 
import numpy as np
import networkx as nx
import sys  mj
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt

In [2]:
df_csv = pd.read_csv("mp_samples.csv")
df_csv_data = df_csv.iloc[:, 2:2971]
df_csv.head()
df_graph = pd.read_csv("graph.csv")
df_graph.head()

,Unnamed: 0,sid,values,class,did,family,order,genus,phylum,kingdom,metaprotein,species,superkingdom
0,0,1,0,Mammalia,382,Hominidae,Primates,Homo,Chordata,Metazoa,CATG_HUMAN Cathepsin G OS=Homo sapiens GN=CTSG...,Homo sapiens,Eukaryota
1,1,1,0,Mammalia,2881,Hominidae,Primates,Homo,Chordata,Metazoa,RP1L1_HUMAN Retinitis pigmentosa 1-like 1 prot...,Homo sapiens,Eukaryota
2,2,1,2,unassigned,2787,unassigned,unassigned,unassigned,Chordata,Metazoa,TTHY_PANTR Transthyretin OS=Pan troglodytes GN...,unassigned,Eukaryota
3,3,1,0,Actinobacteria,2688,Bifidobacteriaceae,Bifidobacteriales,Bifidobacterium,Actinobacteria,unassigned,TIG_BIFAA Trigger factor OS=Bifidobacterium ad...,unassigned,Bacteria
4,4,1,0,Bacteroidia,2589,Bacteroidaceae,Bacteroidales,Bacteroides,Bacteroidetes,unassigned,SUSC_BACTN TonB-dependent receptor SusC OS=Bac...,Bacteroides thetaiotaomicron,Bacteria


In [50]:
df_graph.shape

(142512, 13)

In [3]:
def edge_list_from_graph(df_graph):
    """Function to extract edge list from the whole graph
    
    Args: 
        df_graph (dataframe): dataframe containing the whole graph with all the edges in rows
    """
    df_edge_list = df_graph[['sid', 'did', 'values']]
    df_edge_list['sid'] = 'p' + df_edge_list['sid'].astype(str)
    df_edge_list['did'] = 'mp' + df_edge_list['did'].astype(str)
    
    return df_edge_list

In [4]:
df_edge_list = edge_list_from_graph(df_graph)
df_edge_list_0_dropped = df_edge_list.drop(df_edge_list[df_edge_list.values == 0].index)
df_edge_list_0_dropped

C:\Users\49171\AppData\Local\Temp\ipykernel_14604\992845728.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edge_list['sid'] = 'p' + df_edge_list['sid'].astype(str)
C:\Users\49171\AppData\Local\Temp\ipykernel_14604\992845728.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edge_list['did'] = 'mp' + df_edge_list['did'].astype(str)


,sid,did,values
2,p1,mp2787,2
31,p1,mp2570,15
57,p1,mp2743,7
59,p1,mp2969,31
63,p1,mp2721,17
...,...,...,...
142496,p48,mp283,1
142497,p48,mp410,7
142498,p48,mp62,4
142506,p48,mp185,69


In [5]:
def adj_mat_from_edge_list(df_edge_list):
    """Function to create adjacency matrix of nodes from the list of edges
    
    Args:
        df_edge_list (dataframe): dataframe containing all edges between nodes and corresponding edge weights
    """
    G = nx.MultiGraph()
    values_list = df_edge_list.values.tolist()
    
    for i in range(len(values_list)):
        G.add_edge(values_list[i][0], values_list[i][1], weight=values_list[i][2])
        
    adj_mat = nx.adjacency_matrix(G).A
    
    return G, adj_mat

In [7]:
G, adj_mat = adj_mat_from_edge_list(df_edge_list)
print(adj_mat)

C:\Users\49171\AppData\Local\Temp\ipykernel_14604\2311397839.py:13: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_mat = nx.adjacency_matrix(G).A


[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ... 34 10 13]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0 34  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0 13  0 ...  0  0  0]]


In [8]:
degrees = [val for (node, val) in G.degree()]

In [9]:
len(degrees)

3017

In [10]:
adj_mat.shape

(3017, 3017)

In [11]:
df_cluster = pd.DataFrame(G.nodes, columns = ['nodes'])
df_cluster

,nodes
0,p1
1,mp382
2,mp2881
3,mp2787
4,mp2688
...,...
3012,p44
3013,p45
3014,p46
3015,p47


In [51]:
np.set_printoptions(threshold=np.inf)
df_cluster['cluster'] = SpectralClustering(n_clusters = 4, affinity='precomputed').fit_predict(adj_mat)
df_cluster


C:\Users\49171\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


,nodes,cluster,label
0,p1,1,p_c
1,mp382,2,mp
2,mp2881,3,mp
3,mp2787,2,mp
4,mp2688,1,mp
...,...,...,...
3012,p44,2,p_uc
3013,p45,2,p_uc
3014,p46,2,p_uc
3015,p47,0,p_uc


In [52]:
df_cluster['label'] = ['mp' if 'm' in x else 'p' for x in df_cluster['nodes']]

for index, row in df_cluster[~df_cluster.nodes.str.contains('m')].iterrows():
    
    p = row.nodes.split("p")[1]
    
    if int(p) < 18:
        df_cluster.at[index, 'label'] = 'p_c'
        
    else:
        if int(p) < 29:
            df_cluster.at[index, 'label']  = 'p_cd'
            
        else:
            df_cluster.at[index, 'label']  = 'p_uc'
            
df_cluster
        
        

,nodes,cluster,label
0,p1,1,p_c
1,mp382,2,mp
2,mp2881,3,mp
3,mp2787,2,mp
4,mp2688,1,mp
...,...,...,...
3012,p44,2,p_uc
3013,p45,2,p_uc
3014,p46,2,p_uc
3015,p47,0,p_uc


In [53]:
cluster_p_c = df_cluster[df_cluster['label'] == 'p_c']
cluster_p_cd = df_cluster[df_cluster['label'] == 'p_cd']
cluster_p_uc = df_cluster[df_cluster['label'] == 'p_uc']

In [54]:
cluster_p = df_cluster[df_cluster['label'] == 'p']
cluster_mp = df_cluster[df_cluster['label'] == 'mp']

In [55]:
cluster_0 = df_cluster[df_cluster['cluster'] == 0]
cluster_1 = df_cluster[df_cluster['cluster'] == 1]
cluster_2 = df_cluster[df_cluster['cluster'] == 2]
cluster_3 = df_cluster[df_cluster['cluster'] == 3]

In [56]:
print(cluster_0.label.unique(), cluster_0.shape)
print(cluster_1.label.unique(), cluster_1.shape)
print(cluster_2.label.unique(), cluster_2.shape)
print(cluster_3.label.unique(), cluster_3.shape)

['mp' 'p_uc'] (390, 3)
['p_c' 'mp' 'p_cd' 'p_uc'] (775, 3)
['mp' 'p_cd' 'p_uc'] (1446, 3)
['mp' 'p_c' 'p_cd'] (406, 3)


In [57]:
cluster_0_c = cluster_0[cluster_0['label'] == 'p_c']
cluster_0_cd = cluster_0[cluster_0['label'] == 'p_cd']
cluster_0_uc = cluster_0[cluster_0['label'] == 'p_uc']
cluster_0_mp = cluster_0[cluster_0['label'] == 'mp']

cluster_1_c = cluster_1[cluster_1['label'] == 'p_c']
cluster_1_cd = cluster_1[cluster_1['label'] == 'p_cd']
cluster_1_uc = cluster_1[cluster_1['label'] == 'p_uc']
cluster_1_mp = cluster_1[cluster_1['label'] == 'mp']

cluster_2_c = cluster_2[cluster_2['label'] == 'p_c']
cluster_2_cd = cluster_2[cluster_2['label'] == 'p_cd']
cluster_2_uc = cluster_2[cluster_2['label'] == 'p_uc']
cluster_2_mp = cluster_2[cluster_2['label'] == 'mp']

cluster_3_c = cluster_3[cluster_3['label'] == 'p_c']
cluster_3_cd = cluster_3[cluster_3['label'] == 'p_cd']
cluster_3_uc = cluster_3[cluster_3['label'] == 'p_uc']
cluster_3_mp = cluster_3[cluster_3['label'] == 'mp']

In [58]:
print(cluster_0_c.label.unique(), cluster_0_c.shape)
print(cluster_0_cd.label.unique(), cluster_0_cd.shape)
print(cluster_0_uc.label.unique(), cluster_0_uc.shape)
print(cluster_0_mp.label.unique(), cluster_0_mp.shape)

[] (0, 3)
[] (0, 3)
['p_uc'] (2, 3)
['mp'] (388, 3)


In [59]:
print(cluster_1_c.label.unique(), cluster_1_c.shape)
print(cluster_1_cd.label.unique(), cluster_1_cd.shape)
print(cluster_1_uc.label.unique(), cluster_1_uc.shape)
print(cluster_1_mp.label.unique(), cluster_1_mp.shape)

['p_c'] (16, 3)
['p_cd'] (4, 3)
['p_uc'] (2, 3)
['mp'] (753, 3)


In [60]:
print(cluster_2_c.label.unique(), cluster_2_c.shape)
print(cluster_2_cd.label.unique(), cluster_2_cd.shape)
print(cluster_2_uc.label.unique(), cluster_2_uc.shape)
print(cluster_2_mp.label.unique(), cluster_2_mp.shape)

[] (0, 3)
['p_cd'] (3, 3)
['p_uc'] (16, 3)
['mp'] (1427, 3)


In [61]:
print(cluster_3_c.label.unique(), cluster_3_c.shape)
print(cluster_3_cd.label.unique(), cluster_3_cd.shape)
print(cluster_3_uc.label.unique(), cluster_3_uc.shape)
print(cluster_3_mp.label.unique(), cluster_3_mp.shape)

['p_c'] (1, 3)
['p_cd'] (4, 3)
[] (0, 3)
['mp'] (401, 3)


In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
pos = nx.spring_layout(G)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos)
plt.show()